<a href="https://colab.research.google.com/github/951237/TIL/blob/master/TIL__200601_%EC%BD%94%EB%A1%9C%EB%82%98_%ED%98%84%ED%99%A9_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_feat_%EC%9D%8C%EC%84%B1%EC%A7%80%EC%9B%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyttsx3
!pip install SpeechRecognition

     |████████████████████████████████| 32.8MB 123kB/s 


In [0]:
import requests, json       # 
import pyttsx3      # 음성 글씨로 나타냄
import re, threading, time  # threading 조사하기
import speech_recognition as sr     # 음성인식 라이브러리

In [0]:
# api키 값 - parsehub.com 사이트 
API_KEY = " "   
PROJECT_TOKEN = ""
RUN_TOKEN = ""

In [0]:
# 클래스 생성
class Data():
    # 클래스 초기화
    def __init__(self, api_key, project_token):
        self.api_key = api_key      # api키
        self.project_token = project_token  # 프로젝트 킥
        self.param = {
            "api_key" : self.api_key    # 파라미터 api키
        }
        self.data = self.get_data()     # 데이터 가져와서 데이터에 넣기. 함수가 아래에 정의 되어 있는데, 작동하네
    
    #  함수 - 데이터 가져오기
    def get_data(self):
        response = requests.get(f'https://www.parsehbu.com/api/v2/projects/{self.project_token}/last_ready_run/data', params=self.params)   # 리퀘스트로 내용가져오기
        data = json.loads(response.text)    # 가져온 json 내용을 데이터 변수에 지정
        return Data 

    # 함수 - 전체 확진자 데이터 얻기
    def get_total_cases(self):
        data = self.data['total']       # json파일에서 전체 값 가져오기

        for content in data:        # 반복문
            if content['name'] == 'Coronavirus Cases:':      # json에서 일치하는 값 가져오기
                return content['value']     # 값 리턴

    # 함수 - 전체 사망자 데이터 가져오기
    def get_total_deaths(self):     
        data = self.data['total']       # json에서 total 전체 가져오기

        for content in data:        # 반복문
            if content['name'] == 'Deaths:':    # 사망자 레이블 일치하는 경우
                return content['value']     # 데이터 반환 숫자 반환

        return "0"      # 해당없으면 0으로 반환

    # 함수 나라가져오기
    def get_country_data(self, country):
        data = self.data['country']

        for content in data:
            if content['name'].lower() == country.lower():
                return content
        return "0"

    def get_list_of_countries(self):
        countries = []

        for country in self.data['country']:
            countries.append(country['name'].lower())

    def update_data(self):
        response = requests.post(f'https://www.parsehbu.com/api/v2/projects/{self.project_token}/run', params=self.params)

        def poll():
            time.sleep(0.1)
            old_data = self.data
            while True:
                new_data = self.get_data()
                if new_data != old_data:
                    self.data = new_data
                    print('Data updated')
                    break
            time.sleep(5)

        t = threading.Thread(target=poll)
        t.start()

In [0]:
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
        said = ""

    try:
        said = r.recognize_google(audio)

    except Exception as e:
        print("Exception:", str(e))

    return said.lower()

In [0]:
def main():
    print("Started Programs")
    data = Data(API_KEY, PROJECT_TOKEN)
    END_PHRASE = 'stop'
    country_list = data.get_list_of_countries()

    TOTAL_PATTERNS = {
        re.compile("[\w\s] + total [\w\s] + cases") : data.get_total_cases,
        re.compile("[\w\s] + total cases") : data.get_total_cases,
        re.compile("[\w\s] + total [\w\s] + deaths") : data.get_total_deaths,
        re.compile("[\w\s] + total deaths") : data.get_total_deaths
    }

    COUNTRY_PATTERNS = {
        re.compile("[\w\s] + cases [\w\s] + ") : lambda country : data.get_countries_data(country)['total_cases'],
        re.compile("[\w\s] + deaths [\w\s] + ") : lambda country : data.get_countries_data(country)['total_deaths']
    }

    UPDATE_COMMAND = "update"

    while True:
        print("Listening...")
        text = get_audio()
        print(text)
        result = None

        for pattern, func in COUNTRY_PATTERNS.items():
            if pattern.match(text):
                words = set(text.split(" "))
                for country in country_list:
                    if country in words:
                        result = func(country)
                        break
        
        for pattern, func in TOTAL_PATTERNS.itmes():
            if pattern.match(text):
                result = func()
                break

        if text == UPDATE_COMMAND:
            result = "Data is being updated. This may take a moment."
            data.update_data()

        if result:
            speak(result)

        if text.find(END_PHRASE) != -1:
            print("Exit")
            break
